<a href="https://colab.research.google.com/github/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [5]:
#Importing libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime
from sklearn.preprocessing import MinMaxScaler

In [6]:
df = pd.read_csv("Helpdesk.csv")

In [ ]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21348 non-null  object
 1   lifecycle:transition  21348 non-null  object
 2   org:resource          21348 non-null  object
 3   time:timestamp        21348 non-null  object
 4   Activity              21348 non-null  object
 5   Resource              21348 non-null  object
 6   case:concept:name     21348 non-null  object
 7   case:variant          21348 non-null  object
 8   case:variant-index    21348 non-null  int64 
 9   case:creator          21348 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format='ISO8601')

<ipython-input-245-97bac535b9d0>:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format='ISO8601')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21221 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21221 non-null  object
 1   lifecycle:transition  21221 non-null  object
 2   org:resource          21221 non-null  object
 3   time:timestamp        21221 non-null  object
 4   Activity              21221 non-null  object
 5   Resource              21221 non-null  object
 6   case:concept:name     21221 non-null  object
 7   case:variant          21221 non-null  object
 8   case:variant-index    21221 non-null  int64 
 9   case:creator          21221 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.8+ MB


In [ ]:
df['new_date'] = [d.date() for d in df['time:timestamp']]
df['new_time'] = [d.time() for d in df['time:timestamp']]

In [ ]:
df['time_lapsed'] = df.groupby('case:concept:name')['time:timestamp'].transform('min')

In [ ]:
df['time_lapsed'] = df['time:timestamp'] - df['time_lapsed']

In [ ]:
df['concept:name'].unique()

array(['Assign seriousness', 'Take in charge ticket', 'Resolve ticket',
       'Closed', 'Wait', 'Create SW anomaly', 'Insert ticket',
       'Schedule intervention', 'RESOLVED', 'INVALID', 'VERIFIED',
       'Resolve SW anomaly', 'Require upgrade', 'DUPLICATE'], dtype=object)

In [ ]:
#Checking if we can concept:name and activity is the same as well as the org:resource and the resource, so that we can delete this feild.
df['Checker_1'] = (df['concept:name'] == df['Activity'])
df['Checker_2'] = (df['org:resource'] == df['Resource'])

In [ ]:
#Let's see if it all the same, if True.
df['Checker_2'].unique()

array([ True])

In [ ]:
#Lets drop it since it is the same
df.drop(['concept:name','org:resource','Checker_1','Checker_2','case:creator','time:timestamp','lifecycle:transition'],axis=1, inplace=True)

In [ ]:
#Lets create a stage number for each activity stage of the help desk, this will help us easily identify where the case is in terms of the process

#This is also to immulate the label encoder
#First create a dataframe with stage numbers and the stage names
stage = pd.DataFrame()
stage['Stage_no'] = [1,2,3,4,5,6,7,8,9,9.1,9.2,10]
stage['Activity'] = ['Insert ticket','Assign seriousness','Take in charge ticket','Wait','Require upgrade','Schedule intervention','Resolve ticket','VERIFIED','RESOLVED','INVALID', 'DUPLICATE','Closed']

In [ ]:
df=pd.merge(df,stage,on='Activity', how='left')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21221 entries, 0 to 21220
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Activity            21221 non-null  object 
 1   Resource            21221 non-null  object 
 2   case:concept:name   21221 non-null  object 
 3   case:variant        21221 non-null  object 
 4   case:variant-index  21221 non-null  int64  
 5   new_date            21221 non-null  object 
 6   new_time            21221 non-null  object 
 7   time_lapsed         21221 non-null  object 
 8   Stage_no            21142 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 1.5+ MB


In [ ]:
df.head()

,Activity,Resource,case:concept:name,case:variant,case:variant-index,new_date,new_time,time_lapsed,Stage_no
0,Assign seriousness,Value 2,Case3608,Variant 33,33,2010-01-13,08:40:25,0 days 00:00:00,2.0
1,Take in charge ticket,Value 2,Case3608,Variant 33,33,2010-01-29,08:52:27,16 days 00:12:02,3.0
2,Resolve ticket,Value 2,Case3608,Variant 33,33,2010-01-29,08:52:34,16 days 00:12:09,7.0
3,Closed,Value 5,Case3608,Variant 33,33,2010-02-13,08:52:48,31 days 00:12:23,10.0
4,Assign seriousness,Value 2,Case2748,Variant 1,1,2010-01-13,12:26:04,0 days 00:00:00,2.0


One-hot-Encoding and Preprocessing

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
encoder = OneHotEncoder(sparse_output=False)

In [ ]:
activities = df[['Activity']]
one_hot_encoded = encoder.fit_transform(activities)

In [ ]:
encoded_df = pd.DataFrame(
    one_hot_encoded,
    columns=encoder.get_feature_names_out(['Activity'])
)

In [ ]:
df = pd.concat([df.drop('Activity', axis=1), encoded_df], axis=1)

In [ ]:
df.head()

,Resource,case:concept:name,case:variant,case:variant-index,new_date,new_time,time_lapsed,Stage_no,Activity_Assign seriousness,Activity_Closed,...,Activity_INVALID,Activity_Insert ticket,Activity_RESOLVED,Activity_Require upgrade,Activity_Resolve SW anomaly,Activity_Resolve ticket,Activity_Schedule intervention,Activity_Take in charge ticket,Activity_VERIFIED,Activity_Wait
0,Value 2,Case3608,Variant 33,33,2010-01-13,08:40:25,0 days 00:00:00,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Value 2,Case3608,Variant 33,33,2010-01-29,08:52:27,16 days 00:12:02,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Value 2,Case3608,Variant 33,33,2010-01-29,08:52:34,16 days 00:12:09,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Value 5,Case3608,Variant 33,33,2010-02-13,08:52:48,31 days 00:12:23,10.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Value 2,Case2748,Variant 1,1,2010-01-13,12:26:04,0 days 00:00:00,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.columns

Index(['Resource', 'case:concept:name', 'case:variant', 'case:variant-index',
       'new_date', 'new_time', 'time_lapsed', 'Stage_no',
       'Activity_Assign seriousness', 'Activity_Closed',
       'Activity_Create SW anomaly', 'Activity_DUPLICATE', 'Activity_INVALID',
       'Activity_Insert ticket', 'Activity_RESOLVED',
       'Activity_Require upgrade', 'Activity_Resolve SW anomaly',
       'Activity_Resolve ticket', 'Activity_Schedule intervention',
       'Activity_Take in charge ticket', 'Activity_VERIFIED', 'Activity_Wait'],
      dtype='object')

In [ ]:
activities = [
    'Activity_Assign seriousness', 'Activity_Closed',
    'Activity_Create SW anomaly', 'Activity_DUPLICATE',
    'Activity_INVALID', 'Activity_Insert ticket',
    'Activity_RESOLVED', 'Activity_Require upgrade',
    'Activity_Resolve SW anomaly', 'Activity_Resolve ticket',
    'Activity_Schedule intervention',
    'Activity_Take in charge ticket',
    'Activity_VERIFIED', 'Activity_Wait'
]

In [ ]:
X_activities = df[activities].values

In [ ]:
df["time_lapsed"] = df["time_lapsed"].dt.seconds
df["time_lapsed_seconds"] = df["time_lapsed"].dt.total_seconds()
time_lapsed = df["time_lapsed_seconds"].values.reshape(-1, 1)
scaler = MinMaxScaler()
time_lapsed_scaled = scaler.fit_transform(time_lapsed)

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
timestamps = pd.to_datetime(
    df["new_date"].astype(str) + " " + df["new_time"].astype(str)
)
day_of_week = timestamps.dt.dayofweek.values / 6  # Normalized (0-1)
hour_of_day = timestamps.dt.hour.values / 23      # Normalized (0-1)

In [ ]:
activity_vectors = df[activities].values

In [ ]:
num_features = len(activities) + 3  # Activities + time_lapsed + day + hour
X = np.zeros((len(df), num_features))
X[:, :len(activities)] = activity_vectors
X[:, len(activities)] = time_lapsed_scaled.flatten()
X[:, len(activities)+1] = day_of_week
X[:, len(activities)+2] = hour_of_day

In [ ]:
divisor = df["time_lapsed_seconds"].max()
y_a = df[activities].shift(-1).fillna(0).values
y_t = df["time_lapsed"].shift(-1).fillna(0).values / divisor

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    # 3. Get data for this fold (no manual initialization needed)
    X_train, X_val = X[train_idx], X[val_idx]
    y_a_train, y_a_val = y_a[train_idx], y_a[val_idx]
    y_t_train, y_t_val = y_t[train_idx], y_t[val_idx]

    # 4. Pad sequences (if sequences have variable lengths)
    maxlen = max(len(seq) for seq in X_train)
    X_train = pad_sequences(X_train, maxlen=maxlen, dtype='float32', padding='post')
    X_val = pad_sequences(X_val, maxlen=maxlen, dtype='float32', padding='post')
    print(f"\nFold {fold+1}:")
    print(f"  Training indices: {train_idx[:5]}...{train_idx[-5:]}")
    print(f"  Validation indices: {val_idx[:5]}...{val_idx[-5:]}")
    print(f"  X_train shape: {X_train.shape}, X_val shape: {X_val.shape}")


Fold 1:
  Training indices: [0 1 2 3 4]...[3536 3537 3538 3539 3540]
  Validation indices: [3541 3542 3543 3544 3545]...[7072 7073 7074 7075 7076]
  X_train shape: (3541, 17), X_val shape: (3536, 17)

Fold 2:
  Training indices: [0 1 2 3 4]...[7072 7073 7074 7075 7076]
  Validation indices: [7077 7078 7079 7080 7081]...[10608 10609 10610 10611 10612]
  X_train shape: (7077, 17), X_val shape: (3536, 17)

Fold 3:
  Training indices: [0 1 2 3 4]...[10608 10609 10610 10611 10612]
  Validation indices: [10613 10614 10615 10616 10617]...[14144 14145 14146 14147 14148]
  X_train shape: (10613, 17), X_val shape: (3536, 17)

Fold 4:
  Training indices: [0 1 2 3 4]...[14144 14145 14146 14147 14148]
  Validation indices: [14149 14150 14151 14152 14153]...[17680 17681 17682 17683 17684]
  X_train shape: (14149, 17), X_val shape: (3536, 17)

Fold 5:
  Training indices: [0 1 2 3 4]...[17680 17681 17682 17683 17684]
  Validation indices: [17685 17686 17687 17688 17689]...[21216 21217 21218 21219 212

Model building

In [ ]:
!pip install keras

In [ ]:
#Creating the model architecture

In [ ]:
#Importing the required libriares since the LSTM model will be done with Pytorch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils import get_file
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import BatchNormalization

In [ ]:
temp = df.groupby('case:concept:name')
max_len = 0
for name, groups in temp:
  max_len = max(max_len, len(groups.index))
print(max_len)

15


In [ ]:
num_features = len(df['Stage_no'].unique()) + 13
main_input = Input(shape=(max_len, num_features), name='main_input')

In [ ]:
input_data = df[['new_date','new_time','time_lapsed','Activity_Assign seriousness', 'Activity_Closed',
       'Activity_Create SW anomaly', 'Activity_DUPLICATE', 'Activity_INVALID',
       'Activity_Insert ticket', 'Activity_RESOLVED',
       'Activity_Require upgrade', 'Activity_Resolve SW anomaly',
       'Activity_Resolve ticket', 'Activity_Schedule intervention',
       'Activity_Take in charge ticket', 'Activity_VERIFIED', 'Activity_Wait']]

In [ ]:
#Create the training split

In [ ]:
l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(main_input) # the shared layer
b1 = BatchNormalization()(l1)
l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)
l2_2 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in time prediction
b2_2 = BatchNormalization()(l2_2)
act_output = Dense(len(df['Stage_no'].unique()), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(b2_1)
time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(b2_2)

In [ ]:
model = Model(inputs=[main_input], outputs=[act_output, time_output])

opt = Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, weight_decay=0.004, clipvalue=3)

model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [ ]:
print("X_train dtype:", X_train.dtype)
print("y_a_train dtype:", y_a_train.dtype)
print("y_t_train dtype:", y_t_train.dtype)

X_train dtype: float32
y_a_train dtype: float64
y_t_train dtype: object


In [ ]:
y_t_train

array([Timedelta('0 days 00:00:00.266828331'),
       Timedelta('0 days 00:00:00.266829681'),
       Timedelta('0 days 00:00:00.516853362'), ...,
       Timedelta('0 days 00:00:00.234010569'),
       Timedelta('0 days 00:00:00.278901103'),
       Timedelta('0 days 00:00:00.528924977')], dtype=object)

In [ ]:
model.fit(X_train, {'act_output':y_a_train, 'time_output':y_t_train}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, epochs=500)

ValueError: Invalid dtype: object